<img src="assets/banner.png" width="100%" align="left" style="left">

## RF Automatic Gain Control with PYNQ

We present an Automatic Gain Control (AGC) circuit for control of an analogue/RF amplifier with an interactive GUI exposing its parameters. This example requires an external amplifier (the [VeGA from Nooelec](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf)) and some attenuators.
This very simple design focuses on using the RFSoC's "threshold" feedback signals, allowing us to make decisions based on the true input signal --- even before any mixing or decimation. This lets us ensure that the input signal is occupying the full scale of the ADC, minimising quantisation errors without exceeding the limits of the device. We'll generate various input signals, explore some of the limits of this  AGC algorithm, and practice tweaking our parameters for best performance.
The design of this AGC example and the use of the RF Data Converter's threshold signals will be explored.

## Aims
* Set up an AGC system with an RFSoC and an external amplifier
* Design test signals and capture our system's output at full RF speeds
* Interactively explore the behaviour of an AGC circuit and amplifier
* Learn to tune the AGC parameters for different scenarios and encounter the limits of our amplifier configuration

## Table of Contents

* [1. Introduction](#1.-Introduction)
* [2. ...And we're off!](#go)
* [3. Exploring some scenarios](#scenarios)
* [4. A closer look at the hardware](#hw)
    * [4.1 Power estimation](#power-est)
    * [4.2 Logarithms and exponentials](#log-est)
* [5. Conclusion](#conclusion)

----

## 1. Introduction

Unlike the previous example, our FPGA design will be quite straightforward here. We will rely on the RF Data Converter for our signal monitoring and an [external amplifier](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf) to provide analogue gain.
This means that the system is suitable for automatic gain control of RF signals _before_ they enter the ADC, rather than the previous example's fully digital approach for baseband.
Working in the analogue world means that our goals are a little different too. Instead of providing a constant average power for downstream synchronistation or receiver circuits, we will focus more on matching the RF signal to the characteristics of the ADC. In short, we want to ensure that our signal of interest occupies as much of the ADC's range as possible, without exceeding its upper limit. This will help us minimise quantisation errors and give the downstream design a signal with the best characteristics we can.

Let's take a look at how this hardware design is structured.

<a class="anchor" id="fig-1"></a>
<figure>
    <img src='./assets/aagc_loopback_overview.svg'/>
    <figcaption><b>Figure 1: System overview </b></figcaption>
</figure>

This version of our AGC example introduces the RF Data Converter block and an external amplifier. Apart from these additions, the overall structure remains quite familiar. We'll use PYNQ to craft a test signal in software, send it to the DAC via a DMA channel, and simultaneously receive from the ADC on another DMA channel.

Just before we start to interact with this design, let's take a moment to explore the AGC circuit and the assistance we get from the Data Converter's threshold signals.

## 2. Our threshold-based RF AGC

The ADCs on the RF Data Converter can provide us with two user controlled threshold flags per channel. These give us feedback about the analogue input to the ADCs, before any mixing or decimation. Note there are other flags for overrange and overvoltage events, but we simply ignore them here.

For our purposes, we'll employ the `hysteresis` mode. the hysteresis mode allows us to configure an upper threshold value, a lower threshold value, and a hysteresis time. The output will be set as soon as the analogue input to the ADC exceeds our upper threshold value. It will remain set until the input signal remains below our lower threshold for our entire hysteresis time. Full documentation for all modes can be found in Xilinx's [Product Guide 269](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf). 

Our simple AGC implementation will make use of both user defined thresholds, relying on their hysteresis effects to ignore any short-term dips below our desired amplitude. We are only interested in using the hysteresis mode for masking these short dips _in time_, so we just assign a single threshold's upper and lower values to the same parameter.  We employ both thresholds so we can split the ADC's full range into three regions:

  1. *Low*: Input signal's maximum amplitude is too low. Increases gain.
  2. *Mid*: Input signal's maximum amplitude is within our desired range. Hold gain steady.
  3. *High*: Input signal's maximum amplitude is too high. Reduces gain.

The two threshold flags essentially signal which region the analogue input occupies --- i.e. `00` $\rightarrow$ `Low`, `01` $\rightarrow$ `Mid`, and `11` $\rightarrow$ `High`. Since we are keeping this example design simple and showing the use of the Data Converter's threshold features, we do not use any other information about our RF signal. For a more advanced design, we could combine the thresholding features with the power information from the digital I/Q data, as shown in the [baseband AGC](./1_Baseband_AGC.ipynb) example. For this example, we can see that our three states map quite well to a simple up/down counter to control the gain. Since we have the PYNQ environment to hand, we also expose some parameters for this up/down counter.

  * Attack parameters, for when we need to increase the gain
    + `Attack period`: How many cycles to wait between each increment
    + `Attack step`: How much to increment our state by each period
  * Decay parameters, for when we need to decrease the gain
    + `Decay period`: How many cycles to wait between each decrement
    + `Decay step`: How much to decrement our state by each period

You may wish to configure these two parameter sets independently because the thresholding circuits in hysteresis mode will instantaneously react to a high sample (for `decay`) but will, by necessity, have a higher latency for signalling consecutive low samples (for `attack`). The only other characteristic we need to address with our circuit is the non-linearity of our amplifier's control. We're using the buttons on the VeGA board as a 6-bit digital input. According to [the datasheet](https://www.nooelec.com/store/downloads/dl/file/id/103/product/334/vega_datasheet_revision_1.pdf), this control is approximately linear in dB(!) with noticable deviations at both extremes. Our control should be truly linear (not in dB!) since we only use binary high/low feedback and cannot estimate the error any further. Since the VeGA's digital control is a relatively modest 6-bits, we can just generate a small look-up table to convert our linear control signal to dBs and correct for the non-linearity at the extremes. The extra notebook, ["Appendix A: VeGA Non-linearity"](Appendix_A_VeGA_NonLinearity.ipynb), discusses how we can use Python and SciPy to model the VeGA's response as an equation and then generate a look-up table to ensure our control behaves nearly linearly. 

The full AGC circuit is shown below, with its simplicity largely thanks to the Data Converter's thresholding circuitry.

<a class="anchor" id="fig-2"></a>
<figure>
    <img src='./assets/aagc_block_diagram.svg'/>
    <figcaption><b>Figure 2: Block diagram of our threshold-based AGC</b></figcaption>
</figure>


We present an example scenario to help demonstrate the behaviour of the thresholds and the AGC circuit below. The stem plot here shows the samples received by the ADC. Shown in lock-step below the graph, we see the behaviour of the thresholding flags as controlled by the Data Converter, and the reaction this causes in our AGC circuit. The four important events labelled on the time axis are:

  * **A**: A sample is received in the `high` region, so we want to reduce our gain.
    + _Thresholds_: Current sample exceeds our upper threshold, so upper threshold is immediately set.
    + _Our AGC_: Upper threshold is set, so we want to start decaying our control signal
    
    
  * **B**: We've stayed in the `mid` region for at least `delay` cycles now, so we can hold our gain steady.
    + _Thresholds_: Samples have remained below the upper threshold for the full hysteresis time, so upper threshold is now deasserted.
    + _Our AGC_: Upper threshold is no longer set, so we stop decrementing our control signal
    
    
  * **C**: The last `delay` samples have all been in the `low` region, so we want to start ramping up our gain.
    + _Thresholds_: Samples have remained below the lower threshold for the full hysteresis time, so the lower threshold is now deasserted.
    + _Our AGC_: Lower threshold is no longer set, so we start incrementing our control signal
    
    
  * **D**: A sample is received in the `mid` region, so we can hold our gain steady.
    + _Thresholds_: Current sample exceeds our lower threshold, so lower threshold is immediately set.
    + _Our AGC_: Lower threshold is set, so we stop incrementing our control signal
    

<a class="anchor" id="fig-3"></a>
<figure>
    <img src='./assets/rfdc_threshold_graph.svg'/>
    <figcaption><b>Figure 3: Example of our threshold-based AGC in action </b></figcaption>
</figure>


To conclude our discussion of the hardware design, let's take a moment to compare the response of this threshold-based AGC to that of the baseband AGC [presented previously](./1_Baseband_AGC.ipynb).
This design will prove slightly more challenging to visualise, since we're now working with the full RF sampling data rates. In order to keep the plots reasonably nimble, we'll be looking at a smaller window of time than before. This may make any transient behaviour look a little longer relative to the total buffer --- so be sure to compare the x-axes with the appropriate units!

There are a couple of other unavoidable extra latency due to the structure of the system. Our FPGA clock speed is necessarily less than the speed of the ADCs. Since the AGC control circuit is running in the FPGA it will not be able to react to each individual sample, but consider N samples in parallel. Furthermore, even if the FPGA fabric was running in the GHz range, the ADCs are x8 interleaved so we'd see similar effects anyway! The second factor is the latency introduced by communicating with our external analogue amplifier instead of our digital multiplier.

Finally, the baseband AGC design corrected its gain using the log of the detected error in power. This let it quickly converge on the desired average power for both small and extremely large instantaneous fluctuations in input power. The threshold-based design does not react so intelligently since we are solely using the feedback from the Data Converter's threshold features. We do this to provide a clearer example of how these features work, however there is nothing precluding a design that makes use of both information sources (thresholds *and* baseband I/Q data) simultaneously. In our design, we just step our control signal in a straight line (with a configurable gradient) until we hit the desired input level. We do not change this logarithmically or even linearly with respect to some error estimate since we'd need more information about the input signal to do that. The thresholds are also measure the absolute _amplitude_ of the input signal, rather than the power of the input signal. Still, we present a useful means of performing AGC for a real-world RF signal with an extremely simple circuit.

## 3. Setting up the amplifier

In order to use this demonstration we need to physically set up our VeGA board and include it in a loopback system between our DAC and ADC. Two important points upfront:


  1. **We need to remove the `D0`$\rightarrow$`D5` switches from the VeGA board** in order to drive them digitally.<br/>
     There is no real need for a soldering iron though.
  
  2. **We need some attenuators between the VeGA output and the RFSoC's ADC for safety**.<br/>
     The VeGA board runs from a 5V supply and any signal presented to the RFSoC above 1 $V_{p-p}$ may cause damage to its frontend. These attenuators will help ensure that this voltage mismatch can't damage our new toy... 
     

To remove the `D0`$\rightarrow$`D5` switches, we can choose to desolder them from the board or dismantle them in-place. There is no need to remove the potentiometer, so you will still be able to manually control the gain without the switches (i.e. don't panic, you're not completely breaking your new gadget!). Desoldering the switches obviously requires some extra tools but is easily reversible. Dismantling them doesn't require any extra tools or skill, but reassembling the switches by hand would be _extremely_ tedious. Each button can be dismantled by lifting up the little white tabs visible in the photos below. Make sure to remove the main pole so it cannot make contact with either throw once the white case has been removed.

Finally, ensure that the red "Mode" switch is pressed so the gain control signal will be derived from our switches instead of the potentiometer.

<table style="margin: 0 0;">
<tbody>
<tr>
    <td>
        <a class="anchor" id="fig-4a"></a>
        <figure>
        <img src='./assets/vega_setup/vega_with_sw.JPG'/ style="max-width:90%;max-height:500px;" />
        <figcaption><b>Figure 4a: VeGA board with original switches</b></figcaption>
        </figure>
    </td>
    <td>
        <a class="anchor" id="fig-4b"></a>
        <figure>
        <img src='./assets/vega_setup/vega_without_sw.JPG'/ style="max-width:90%;max-height:500px;" />
        <figcaption><b>Figure 4b: VeGA board with dismantled switches (also shows the potentiometer removed but this is not recommended)</b></figcaption>
        </figure>
    </td>
</tr>
</tbody>
</table>


Now that we can control the VeGA via its header, let's connect it to the RFSoC. We present the wiring diagram below --- showing the RFSoC's PMOD connector controlling the VeGA's gain and the loopback path including the VeGA and 16 dB attenuation. Again, this **attenuation is important** to avoid damage to the RFSoC's analogue frontend. We're relying on this to reduce the VeGA's $0\rightarrow5\ V$ range to well within the ADC's $1\ V_{p-p}$ requirements.

<a class="anchor" id="fig-5"></a>
<figure>
    <img src='./assets/vega_zcu111_setup.svg'/>
    <figcaption><b>Figure 5: Wiring diagram between VeGA and ZCU111</b></figcaption>
</figure>

For completeness, we also give photos demonstrating this setup below. Figure 6a shows the VeGA header being connected to the ZCU111's PL PMOD 1 port (connector J49) and Figure 6b shows the two SMA cables for our loopback:

  1. DAC 229 Tile 1 Block 2 $\rightarrow$ VeGA input
  2. VeGA output $\rightarrow$ -16 dB $\rightarrow$ ADC 224 Tile 0 Block 0. 

<table style="margin: 0 0;">
<tbody>

<tr>
    <td>
        <a class="anchor" id="fig-6a"></a>
        <figure>
        <img src='./assets/vega_setup/zcu111_vega_to_pmod.JPG'/ style="max-width:90%;max-height:500px;" />
        <figcaption><b>Figure 6a: Connecting the VeGA switches header to the PMOD header</b></figcaption>
        </figure>
    </td>
    <td>
        <a class="anchor" id="fig-6b"></a>
        <figure>
        <img src='./assets/vega_setup/zcu111_smas.JPG'/ style="max-width:90%;max-height:500px;" />
        <figcaption><b>Figure 6b: Connecting the VeGA RF input & output to the RFSoC</b></figcaption>
        </figure>
    </td>
</tr>
</tbody>
</table>


## 2. ...And we're off! <a class="anchor" id="go"></a>

Run the following code cell to launch our GUI. This lets you set AGC parameters and generate your own input signals. The input signals will automatically be streamed through the RF Data Converters and visualised below.
Note that the DAC and ADC blocks are both running at 1.024 GHz with bypassed mixer and no decimation/interpolation. This means we are visualising our RF signal as directly as possible after sampling.

You can control the envelope of the input signal by clicking and dragging the "envelope handles" (the blue circles). The hysteresis time parameter (how long we wait for consecutive cycles to be under before deasserting a threshold) is shown on the output plot as a grey area at the far left.

Check [this animation](https://github.com/strath-sdr/pynq_agc/blob/master/demonstration.gif) to see an example of setting the envelope.

In [ ]:
import pynq_agc
from pynq import Overlay
import plotly.express as px

ol = Overlay('block_design.bit')

In [ ]:
ol.rf_agc.gui()

You can chose from a selection of preset scenarios using the dropdown list at the bottom of this GUI. We recommend exporing these first and we discuss them below. 

## 3. Exploring some scenarios

Here are a few examples of configurations which demonstrate some basic tests for our RF AGC.
You can select each scenario the with "Preset Scenarios" input at the bottom of the GUI.

> ### Offset
> Here we have an effect that emulates "slow fading" of a simple signal. We'd usually expect this to be caused by movement, often with buildings or terrain impeding the signal. If the receiver is sensitive to the amplitude of the received signal (e.g. AM or many QAM synchronisers), we'd need use an AGC to compensate for the changes in signal strength.
>
> Try different window and alpha parameters to see how constant you can keep the output power. Notice that increasing the window size will reduce the time resolution of our gain control, which leads to a noticeable saw-tooth envelope for slow fading effects. If alpha is too high, we can see the output ringing where we've overshot by adjusting the gain too aggressively. If alpha is too low, we might not be compensating quickly enough for the fading effects.



## 5. Conclusion <a class="anchor" id="conclusion"></a>

We've presented an AGC design for use at RF, making use of the Data Converter's thresholding features. We've provided an interactive GUI to help _gain_ an intuitive understanding of how this AGC and the threshold features behave. We've discussed some example scenarios which highlight common types of correction our AGC will have to make. While we have shied away from the mathematical complexity of the previous design, we focus instead on interacting with the Data Converters and the VeGA board in order to perform gain correction in the analogue domain. This is especially useful for keeping our signal within a good range for the ADC; protecting the device at one extreme and minimising quantisation error at the other.

This has hopefully given a good foundation for control of a Variable Gain Amplifier from the RFSoC. Perhaps this has even inspired you to go a little further, for example, combining our two approaches for a better overall response!

## References <a class="anchor" id="references"></a>

[1] - [Tony Rouphael, "RF and Digital Signal Processing for Software-Defined Radio"](https://www.elsevier.com/books/rf-and-digital-signal-processing-for-software-defined-radio/rouphael/978-0-7506-8210-7)

[2] - [EDN, "Wireless 101: Automatic Gain Control (AGC)", Accessed: 09/02/2021](https://www.edn.com/wireless-101-automatic-gain-control-agc/)

[3] - [Grant Griffin, DSPGuru, "DSP Trick: Magnitude Estimator", Accessed: 09/02/2021](https://dspguru.com/dsp/tricks/magnitude-estimator/)

[4] - [Yevgen Voronenko, SPIRAL Project, "Multiplier Block Generator", Accessed: 09/02/2021](http://spiral.ece.cmu.edu/mcm/gen.html)

[5] - [Pete Johnson, Beyond Circuits, "Computing the Logarithm Base 2", Accessed: 22/02/2021](https://www.beyond-circuits.com/wordpress/2015/10/computing-the-logarithm-base-2/)

[6] - [Jack Volder, "The CORDIC computing technique", Accessed: 22/02/2021](https://dl.acm.org/doi/10.1145/1457838.1457886)

[7] - [J. S. Walther, "A unified algorithm for elementary functions", Accessed: 22/02/2021](https://dl.acm.org/doi/10.1145/1478786.1478840)

[8] - [Adam Walker, "clash-utils" GitHub Repository, Accessed: 22/02/2021](https://github.com/adamwalker/clash-utils)

## Revision History
* **v0.2** | *Craig Ramsay* | 22/09/2021 | Added scenarios and notebook discussion
* **v0.1** | *Craig Ramsay* | 09/02/2021 | First alpha demo

[⬅️ Previous Notebook](1_Baseband_AGC.ipynb) | | [Next Notebook ➡️](Appendix_A_VeGA_NonLinearity.ipynb)